# 이미지 증폭을 위한 함수

In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# 이미지 회전
def rotate_image(image, angle):
    return image.rotate(angle, fillcolor=(255,255,255))

# 이미지 크기 조정
def scale_image(image, scale_factor):
    width, height = image.size
    new_size = (int(width * scale_factor), int(height * scale_factor))
    return image.resize(new_size)

# 이미지 좌우 반전
def horizontal_flip(image):
    return image.transpose(method=Image.FLIP_LEFT_RIGHT)

# 이미지 상하 반전
def vertical_flip(image):
    return image.transpose(method=Image.FLIP_TOP_BOTTOM)

# 이미지 잘라내기기
def crop_image(image, crop_size):
    width, height = image.size
    left = (width - crop_size[0]) / 2
    top = (height - crop_size[1]) / 2
    right = (width + crop_size[0]) / 2
    bottom = (height + crop_size[1]) / 2
    return image.crop((left, top, right, bottom))

# 이미지 색상 변경
def color_jitter(image, brightness=1.0, contrast=1.0):
    return Image.eval(image, lambda x: int(x * brightness))

# 이미지에 노이즈 추가
def add_noise(image):
    noise = np.random.normal(0, 25, (image.size[1], image.size[0], 3)).astype(np.uint8)
    noisy_image = np.array(image) + noise
    return Image.fromarray(np.clip(noisy_image, 0, 255).astype(np.uint8))

# 이미지에 기울기 설정
def shearing(image, shear_factor):
    width, height = image.size
    matrix = (1, shear_factor, 0, 0, 1, 0)
    return image.transform((width, height), Image.AFFINE, matrix, fillcolor=(255, 255, 255))

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
# 파일 경로
IMG_DIR = '../poketmon2nd_sil/'

In [ ]:
## 현재 모든 이미지 파일 리스트
allitems = os.listdir(IMG_DIR)
filelist = []

for item in allitems:
    # 파일과 폴더 체크 후 파일만 추출
    # print(os.path.isfile(IMG_DIR+item))
    if os.path.isfile(IMG_DIR+item):
        filelist.append(IMG_DIR+item)

print(f'이미지파일리스트 : {len(filelist)}개 \n {filelist}')

In [ ]:
image = cv2.imread('../poketmon2nd_sil/025102.png')

a = [(color_jitter(image, i)) for i in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.]]

In [ ]:
import cv2
import numpy as np

# filelist = [r'E:\AI_KDT7\project\poketmon2nd_sil\000101.png', r'E:\AI_KDT7\project\poketmon2nd_sil\000101.png']  # 이미지 파일 경로 리스트
output_folder = '../poketmon2nd_sil_white'

for i in filelist:
    img = cv2.imread(fr'{i}', cv2.IMREAD_UNCHANGED)
    if img is None:
        print("이미지를 읽을 수 없습니다. 경로를 확인하세요.")
    else:
        # 알파 채널 확인 (투명도)
        if img.shape[2] == 4:  # RGBA 형식일 경우
            # 알파 채널을 분리
            b, g, r, a = cv2.split(img)

            # 알파 채널을 사용하여 객체만 남기기
            img_rgb = cv2.merge((b, g, r))  # BGR 형식으로 변환
            mask = a > 0  # 알파 채널이 0이 아닌 부분 마스크 생성

            # 마스크를 사용하여 배경을 흰색으로 변경
            output = np.ones_like(img_rgb) * 255  # 흰색 배경
            output[mask] = img_rgb[mask]  # 객체 부분을 흰색 배경에 오버레이
            
            # RGB 이미지로 저장 (알파 채널 제거)
            output_rgb = cv2.cvtColor(output, cv2.COLOR_BGRA2BGR)  # BGR -> RGB 변환
            
            # 결과 이미지 파일로 저장
            output_filename = os.path.join(output_folder, f'output_{os.path.basename(i).split(".")[0]}.png')  # 지정한 폴더에 저장
            cv2.imwrite(output_filename, output_rgb)  # 결과 이미지 저장
            print(f"{output_filename}에 저장되었습니다.")
        else:
            print(f"{i}는 RGBA 형식이 아닙니다.")

In [ ]:
## 현재 모든 이미지 파일 리스트

IMG_DIR = '../poketmon2nd_sil_white/'
allitems = os.listdir(IMG_DIR)
filelist = []

for item in allitems:
    # 파일과 폴더 체크 후 파일만 추출
    # print(os.path.isfile(IMG_DIR+item))
    if os.path.isfile(IMG_DIR+item):
        filelist.append(IMG_DIR+item)

print(f'이미지파일리스트 : {len(filelist)}개 \n {filelist}')

In [ ]:
for i in filelist:

# 이미지 로드
# image_path = './project/poketmonster_background_white/000101.png'
# image = Image.open(image_path)

    image = Image.open(f'{i}') 

    
    # 증강 기법 적용
    augmented_images = [
        scale_image(image, 3.0),
        horizontal_flip(image),
        vertical_flip(image),
        add_noise(image)
    ]

    # 회전한 이미지 35장
    [augmented_images.append(rotate_image(image, i)) for i in range(10,360, 10)]

    # 이미지 기울기 설정 => 0.3 ~ 0.5가 가장 잘 나옴
    [augmented_images.append(shearing(image, i)) for i in [0.3,0.4,0.5]]

    # 이미지 밝기 조정 0.5 ~ 2.0 => 0.2씩 증강
    [augmented_images.append(color_jitter(image, i)) for i in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4]]

    # # 이미지 잘라내기기 => 해당 이미지는 망가지는 것이 많기 때문에 기각각
    # [augmented_images.append(crop_image(image, i)) for i in [(100,100),(50,50),(150,150)]]

    # 결과 표시
    plt.figure(figsize=(12, 8))

    for k, img in enumerate(augmented_images):
        plt.subplot(10,10, k + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()

In [11]:
for i in filelist:

# 이미지 로드
# image_path = './project/poketmonster_background_white/000101.png'
# image = Image.open(image_path)

    image = Image.open(f'{i}') 

    
    # 증강 기법 적용
    augmented_images = [
        scale_image(image, 3.0),
        horizontal_flip(image),
        vertical_flip(image),
        add_noise(image)
    ]

    # 회전한 이미지 35장
    [augmented_images.append(rotate_image(image, i)) for i in range(10,360, 10)]

    # 이미지 기울기 설정 => 0.3 ~ 0.5가 가장 잘 나옴
    [augmented_images.append(shearing(image, i)) for i in [0.3,0.4,0.5]]

    # 이미지 밝기 조정 0.5 ~ 2.0 => 0.2씩 증강
    [augmented_images.append(color_jitter(image, i)) for i in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4]]

    # # 이미지 잘라내기기
    # [augmented_images.append(crop_image(image, i)) for i in [(100,100),(50,50),(150,150)]]


        # 저장할 경로 설정
    for idx, augmented_image in enumerate(augmented_images):
        # 파일 경로 생성
        file_name = f'augmented_image_{i.split("/")[-1].split(".")[0]}_{idx}.png'
        file_path = os.path.join('../poketmon2nd_sil_white_plus', file_name)
        
        # 이미지 저장
        augmented_image.save(file_path)

In [ ]:
import cv2
import os
import csv

# 이미지 파일 경로
image_directory = 'E:\\AI_KDT7\\project\\poketmon_three'
# 저장할 CSV 파일 경로
csv_file_path = 'E:\\AI_KDT7\\project\\poketmon_three\\images_data.csv'

# 이미지 파일 리스트
filelist = [f for f in os.listdir(image_directory) if f.endswith('.png')]

# CSV 파일 열기 (쓰기 모드)
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # CSV 헤더 작성
    header = ['filename'] + [f'pixel_{i}' for i in range(150 * 150 * 3)]  # 150x150 이미지, RGB 채널
    csv_writer.writerow(header)

    for filename in filelist:
        # 이미지 로드
        image_path = os.path.join(image_directory, filename)
        image = cv2.imread(image_path)

        # 이미지가 정상적으로 로드되었는지 확인
        if image is not None:
            # 이미지를 1차원 배열로 변환
            image_flattened = image.flatten()  # (150*150*3,)
            # 파일명과 픽셀 데이터를 CSV에 추가
            csv_writer.writerow([filename] + image_flattened.tolist())
        else:
            print(f"Warning: {filename} could not be loaded.")

print(f"Images saved to CSV file at: {csv_file_path}")


In [ ]:
import cv2

# 이미지 경로
a = r'E:\AI_KDT7\project\poketmon_three\augmented_image_015101_14.png'

# 이미지 로드
b = cv2.imread(a)

# 이미지가 정상적으로 로드되었는지 확인
if b is not None:
    # 이미지 리사이즈
    resized_image = cv2.resize(b, (70, 70))  # (너비, 높이)

    # 리사이즈된 이미지의 shape 출력
    print(resized_image.shape)  # (70, 70, 3)

    # 리사이즈된 이미지를 파일로 저장 (선택 사항)
    cv2.imwrite(r'E:\AI_KDT7\project\poketmon_three\augmented_image_resized.png', resized_image)

else:
    print("Warning: Image could not be loaded.")


In [ ]:
## 현재 모든 이미지 파일 리스트

IMG_DIR = '../poketmon2nd_sil_white_plus/'
allitems = os.listdir(IMG_DIR)
filelist = []

for item in allitems:
    # 파일과 폴더 체크 후 파일만 추출
    # print(os.path.isfile(IMG_DIR+item))
    if os.path.isfile(IMG_DIR+item):
        filelist.append(IMG_DIR+item)

print(f'이미지파일리스트 : {len(filelist)}개 \n {filelist}')

In [ ]:
import cv2
import os

# 이미지 파일 경로
image_directory = '../poketmon2nd_sil_white_plus'
# 리사이즈된 이미지를 저장할 경로
resize_directory = '../poketmon2nd_sil_white_plus_resize'

# 이미지 파일 리스트
filelist = [f for f in os.listdir(image_directory) if f.endswith('.png')]
filelist = [os.path.join(image_directory, f) for f in filelist]  # 전체 경로 추가

# # 저장할 경로가 존재하지 않으면 생성
# os.makedirs(resize_directory, exist_ok=True)

for i in filelist:
    # 이미지 로드
    img = cv2.imread(i)

    # 이미지가 정상적으로 로드되었는지 확인
    if img is not None:
        # 이미지 리사이즈
        resized_image = cv2.resize(img, (100,100))  # (너비, 높이)

        # 저장할 파일 이름 생성
        filename = os.path.basename(i)  # 파일명만 추출
        resized_image_path = os.path.join(resize_directory, filename)  # 저장할 전체 경로

        # 리사이즈된 이미지 저장
        cv2.imwrite(resized_image_path, resized_image)
    else:
        print(f"Warning: {i} could not be loaded.")

print(f"Resized images saved to: {resize_directory}")


In [ ]:
import cv2
import os
import csv

# 이미지 파일 경로
image_directory = '../poketmon2nd_sil_white_plus_resize'
# 저장할 CSV 파일 경로
csv_file_path = '../poketmon2nd_sil_white_plus_resize/images_data.csv'

# 이미지 파일 리스트
filelist = [f for f in os.listdir(image_directory) if f.endswith('.png')]

# CSV 파일 열기 (쓰기 모드)
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # CSV 헤더 작성
    header = ['filename'] + [f'pixel_{i}' for i in range(100 * 100* 3)]  
    csv_writer.writerow(header)

    for filename in filelist:
        # 이미지 로드
        image_path = os.path.join(image_directory, filename)
        image = cv2.imread(image_path)

        # 이미지가 정상적으로 로드되었는지 확인
        if image is not None:
            # 이미지를 1차원 배열로 변환
            image_flattened = image.flatten()
            # 파일명과 픽셀 데이터를 CSV에 추가
            csv_writer.writerow([filename] + image_flattened.tolist())
        else:
            print(f"Warning: {filename} could not be loaded.")

print(f"Images saved to CSV file at: {csv_file_path}")


In [ ]:
import os
import cv2
import csv

# 이미지 파일 경로
image_directory = '../poketmon2nd_sil_white_plus_resize'
# 저장할 CSV 파일 경로
csv_file_path = '../poketmon2nd_sil_white_plus_resize/images_data.csv'


# 이미지 파일 리스트
filelist = [f for f in os.listdir(image_directory) if f.endswith('.png')]

# CSV 파일 열기 (쓰기 모드)
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # CSV 헤더 작성
    header = ['pixel'] * (100*100*3) + ['target']  # 각 픽셀을 'pixel'로 통합
    csv_writer.writerow(header)

    for filename in filelist:
        # 이미지 로드
        image_path = os.path.join(image_directory, filename)
        image = cv2.imread(image_path)

        # 이미지가 정상적으로 로드되었는지 확인
        if image is not None:
            # 파일 이름을 문자열로 변환
            filename = str(filename)

            # 파일 이름 길이 확인
            if len(filename) >= 20:  # 'augmented_image_' (16자) + '015101' (6자) + '.png' (4자)
                target = filename[23:29]  # '015101' 부분
                
                # 이미지를 1차원 배열로 변환
                image_flattened = image.flatten()  
                
                # 파일명, 픽셀 데이터 및 타겟을 CSV에 추가
                csv_writer.writerow(image_flattened.tolist() + [target])

            else:
                print(f"Warning: Filename '{filename}' is too short for target extraction.")
        else:
            print(f"Warning: {filename} could not be loaded.")

print(f"Images and targets saved to CSV file at: {csv_file_path}")

## 원본 이미지 정보 확인 

In [17]:
asd = cv2.imread('../poketmon2nd_sil_white/output_015001.png')

asd.shape

(222, 160, 3)

# 모델링

In [5]:
import pandas as pd

a = pd.read_csv('../poketmon2nd_sil_white_plus_resize/images_data.csv')
a.info

<bound method DataFrame.info of        pixel  pixel.1  pixel.2  pixel.3  pixel.4  pixel.5  pixel.6  pixel.7  \
0        247      246      247      249      247      249      249      249   
1        248      250      251      246      248      248      246      248   
2        255      255      255      255      255      255      253      253   
3        255      255      255      255      255      255      255      255   
4        255      255      255      255      255      255      255      255   
...      ...      ...      ...      ...      ...      ...      ...      ...   
16343    255      255      255      255      255      255      255      255   
16344    255      255      255      255      255      255      255      255   
16345    255      255      255      255      255      255      255      255   
16346    255      255      255      255      255      255      255      255   
16347    255      255      255      255      255      255      255      255   

       pixel.8  pix

In [ ]:
import cv2
import numpy as np
import pandas as pd

# 예시로 DataFrame 생성 (실제 데이터 로드)
# a = pd.read_csv('your_file.csv')  # 실제 CSV 파일 로드

# DataFrame의 내용 확인
print(a.head())

# 첫 번째 행 선택 후 NumPy 배열로 변환
try:
    pixel = a.iloc[0].to_numpy()[:-1]  # 마지막 요소 제거
    pixel1 = pixel.reshape((100, 100, 3))  # 100x100 RGB 이미지로 reshape

    # 데이터 타입을 uint8로 변환
    pixel1 = pixel1.astype(np.uint8)

    # 이미지 표시
    cv2.imshow('sibal', pixel1)
    cv2.waitKey(0)  # 키 입력 대기
    cv2.destroyAllWindows()  # 모든 창 닫기

    
except KeyError as e:
    print(f"KeyError 발생: {e}")
except ValueError as e:
    print(f"ValueError 발생: {e}")
except Exception as e:
    print(f"오류 발생: {e}")


   pixel  pixel.1  pixel.2  pixel.3  pixel.4  pixel.5  pixel.6  pixel.7  \
0    247      246      247      249      247      249      249      249   
1    248      250      251      246      248      248      246      248   
2    255      255      255      255      255      255      253      253   
3    255      255      255      255      255      255      255      255   
4    255      255      255      255      255      255      255      255   

   pixel.8  pixel.9  ...  pixel.29991  pixel.29992  pixel.29993  pixel.29994  \
0      249      249  ...          255          255          255          255   
1      249      247  ...          255          255          255          255   
2      253      249  ...          255          255          255          255   
3      255      255  ...          255          255          255          255   
4      255      255  ...          255          255          255          255   

   pixel.29995  pixel.29996  pixel.29997  pixel.29998  pixel.29999  

In [ ]:
import numpy as np
import cv2

def restore_image_from_row(row):
    # 1. 행을 numpy 배열로 변환
    pixel_data = np.array(row[:-1])  # 마지막 열은 타겟이므로 제외
    pixel_data = pixel_data.reshape((100, 100, 3))  # 100x100x3 형태로 변환

    # 2. 이미지 표시
    cv2.imshow('Restored Image', pixel_data)
    cv2.waitKey(0)  # 키 입력 대기
    cv2.destroyAllWindows()  # 모든 창 닫기

In [14]:
image = cv2.imread('E:\AI_KDT7\project\pocket.png')
cv2.imshow('sbc',image)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터프레임 a가 이미 정의되어 있다고 가정합니다.
# feature_df는 픽셀 데이터, target_sr은 타겟 레이블입니다.
feature_df = a.iloc[:, :-1]  # 마지막 열 제외 (픽셀 데이터)
target_sr = a.iloc[:, -1]    # 마지막 열 (타겟)

# Stratified Split
X_train, X_test, y_train, y_test = train_test_split(feature_df, target_sr,
                                                    test_size=0.2, random_state=42, stratify=target_sr)

# 결과 출력
print(f"X_train => {X_train.ndim}D {X_train.shape} / X_test => {X_test.ndim}D, {X_test.shape}")
print(f"y_train => {y_train.ndim}D {y_train.shape}, / y_test => {y_test.ndim}D, {y_test.shape}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# 랜덤 포레스트 분류기 생성
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# 모델 훈련
rf_classifier.fit(X_train, y_train)

# 예측
y_pred = rf_classifier.predict(X_test)

# 결과 평가
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [31]:
import joblib  # joblib 라이브러리 임포트

# 모델 저장
joblib.dump(rf_classifier, 'rf_model.pkl')  # 파일 이름을 rf_model.pkl로 변경

# 모델 불러오기
# loaded_model = joblib.load('rf_model.pkl')  # 파일 이름도 변경된 것을 사용

['rf_model.pkl']

In [3]:
import cv2

image = cv2.imread('../poketmon2nd_sil_white_plus_resize/augmented_image_output_000101_11.png',cv2.IMREAD_UNCHANGED)
image.shape

(100, 100, 3)

In [67]:
image = cv2.imread('../poketmon2nd_sil/000101.png',cv2.IMREAD_UNCHANGED)
image.shape

(474, 351, 4)

In [63]:
import cv2
import numpy as np

# 이미지 읽기
image = cv2.imread('../val/val (89).png', cv2.IMREAD_UNCHANGED)

# 이미지가 정상적으로 읽혔는지 확인
if image is None:
    print("이미지를 읽는 데 실패했습니다.")
else:
    # 원본 이미지의 형태 확인
    original_shape = image.shape
    print("원본 이미지 형태:", original_shape)

    # 이미지를 1차원 배열로 변환
    image_1d = image.reshape(-1)

    # 3차원으로 변환 (원래의 높이, 너비, 채널 수를 사용)
    image_3d = image_1d.reshape(original_shape)  # 원본 형태로 다시 변환

    # 변환된 3차원 배열의 형태 확인
    print("변환된 3차원 배열 형태:", image_3d.shape)


원본 이미지 형태: (1392, 1024, 4)
변환된 3차원 배열 형태: (1392, 1024, 4)


In [ ]:
import cv2
import numpy as np


image = cv2.imread('../poketmon2nd_sil_white_plus_resize/augmented_image_output_000101_11.png',cv2.IMREAD_UNCHANGED)
image.shape

# 이미지 리사이즈
resized_image = cv2.resize(image, (100, 100))  # 100x100 크기로 리사이즈

# 이미지 평탄화 (1차원 배열로 변환)
flattened_image = resized_image.flatten()  # 1차원 배열로 변환

# 결과 확인
print(flattened_image.shape)  # (30000,) 이 출력됩니다. (100*100*3)


(40000,)


In [5]:
image = cv2.imread('../poketmon2nd_sil_white_plus_resize/augmented_image_output_000101_11.png',cv2.IMREAD_UNCHANGED)
flattened_image = image.flatten()

In [4]:
import joblib 

loaded_model = joblib.load('./rf_model.pkl')

In [6]:
import cv2
import numpy as np

# 예측
prediction = loaded_model.predict([flattened_image])
print(prediction)


[101]


c:\Users\kdp\anaconda3\envs\ML_CV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [3]:
import cv2

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    
    ret, frame = cap.read()
    if not ret:
        print("프레임을 읽을 수 없습니다.")
        break

    # 사각형 그리기 (x, y, 너비, 높이)
    start_point = (266, 188)  # 사각형의 시작점 -> 좌상단
    end_point = (366, 288)    # 사각형의 끝점 -> 우하단
    
    color = (255, 255, 255)       # 색상 
    thickness = 2             # 선 두께

    # 프레임 위에 사각형 
    cv2.rectangle(frame, start_point, end_point, color, thickness)

    # 사각형 내부 이미지 잘라내기
    roi = frame[start_point[1]:end_point[1], start_point[0]:end_point[0]]

    # 결과 프레임 보기
    cv2.imshow('Camera Feed', frame)

    # 's' 키를 눌러 사각형 안의 내용을 저장
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite('../pocket.png', roi)  # 사각형 안의 내용을 이미지로 저장
        print("이미지가 저장되었습니다.")

    print('./')
    # 결과 프레임 보기
    cv2.imshow('Camera Feed', frame)

    # 'q' 키를 눌러 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 모든 창 닫기
cap.release()
cv2.destroyAllWindows()


./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
./
.

In [7]:
import cv2
print(cv2.__version__)


4.11.0
